In [1]:
import requests
import json
import logging
import time, datetime

from enum import Enum

In [2]:
Currency = Enum('Currency', 'EUR USD')
Cryptomoney = Enum('Cryptomoney', 'BTC ETH')
Price = Enum('Price', 'buy sell spot')
Environment = Enum('Environment', 'TEST PROD')

In [3]:
ENV = Environment.TEST

In [4]:
def create_logger(file = 'activity.log', level = logging.DEBUG):
    logger = logging.getLogger()
    logger.setLevel(level)
    # Define logging format
    formatter = logging.Formatter('%(asctime)s :: %(levelname)s :: %(message)s')
    # Create the file handler
    file_handler = logging.handlers.RotatingFileHandler(file, 'a', 1000000, 1)
    file_handler.setLevel(logging.DEBUG)
    file_handler.setFormatter(formatter)
    logger.addHandler(file_handler)
    # Create the stream handler
    stream_handler = logging.StreamHandler()
    stream_handler.setLevel(logging.DEBUG)
    logger.addHandler(stream_handler)
    return logger
    
LOGGER = create_logger()

In [5]:
def get_current_timestamp():
    return datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')

if ENV == Environment.TEST:
    get_current_timestamp()

'2017-12-25 02:01:55'

In [6]:
def initialize_files(logger=None):
    if logger != None:
        logger.warning("Erasing and Reinitializing all CSVs")
    for cmy in [c.name for c in Cryptomoney]:
        for ccy in [c.name for c in Currency]:
            f = open('data/{}-{}.csv'.format(cmy, ccy), 'w')
            f.write('time, ' + ', '.join([p.name for p in Price]) + '\n')
            f.close()

if ENV == Environment.TEST:
    initialize_files(LOGGER)

Erasing and Reinitializing all CSVs


In [7]:
def get_value(cmy = Cryptomoney.BTC, ccy = Currency.EUR, prc = Price.spot):
    url = 'https://api.coinbase.com/v2/prices/{}-{}/{}'.format(cmy.name, ccy.name, prc.name)
    xhr = requests.get(url)
    if xhr.status_code != 200:
        raise Exception('XHR status '+str(xhr.status_code)+': '+xhr.reason+' ('+xhr.url+')')
    response = xhr.json()
    return float(response['data']['amount'])

if ENV == Environment.TEST:
    get_value()

Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/spot HTTP/1.1" 200 83


In [8]:
def get_all_values(cmy = Cryptomoney.BTC, ccy = Currency.EUR):
    results = {}
    for p in Price:
        results[p.name] = get_value(cmy, ccy, p)
    return results

if ENV == Environment.TEST:
    get_all_values()

Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/buy HTTP/1.1" 200 None
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/sell HTTP/1.1" 200 None
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/spot HTTP/1.1" 200 83


In [9]:
def add_line(logger = None):
    for cmy in Cryptomoney:
        for ccy in Currency:
            ts = get_current_timestamp()
            if logger != None:
                logger.info(ts + " - Getting all values.")
            all_values = get_all_values(cmy, ccy)
            f = open('data/{}-{}.csv'.format(cmy.name, ccy.name), 'a')
            f.write(ts + ', ' + ', '.join([str(all_values[p.name]) for p in Price]) + '\n')
            f.close()

if ENV == Environment.TEST:
    add_line(LOGGER)

2017-12-25 02:02:02 - Getting all values.
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/buy HTTP/1.1" 200 None
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/sell HTTP/1.1" 200 None
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-EUR/spot HTTP/1.1" 200 83
2017-12-25 02:02:03 - Getting all values.
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-USD/buy HTTP/1.1" 200 None
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-USD/sell HTTP/1.1" 200 83
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/BTC-USD/spot HTTP/1.1" 200 83
2017-12-25 02:02:03 - Getting all values.
Starting new HTTPS connection (1): api.coinbase.com
https://api.coinbase.com:443 "GET /v2/prices/ETH-EUR/buy

# ----- MAIN -----
(run every hour)

In [ ]:
add_line(LOGGER)